<img src="./images/build_ai_job_application_helper_app.png" width="800" />

# Welcome to the _AI job application helper app_!

This Jupyter Notebook is from [Tampa Bay Artificial Intelligence Meetup’s](https://www.meetup.com/tampa-artificial-intelligence-meetup/) exercise from [our session on Monday, March 17, 2025](https://www.meetup.com/tampa-artificial-intelligence-meetup/events/306353685/?eventOrigin=group_upcoming_events), which took place at [Embarc Collective](https://embarccollective.com/) in [Tampa](https://en.wikipedia.org/wiki/Tampa,_Florida). It was organized by [Joey de Villa](https://www.linkedin.com/in/joeydevilla/) and [Anitra Pavka](https://www.linkedin.com/in/anitrapavka/).

In that session, we demonstrated how to build an application to help people with the job search process. Feel free to use it in your job search, and feel free to modify it to suit your job search needs!

## The job-seeker’s challenge

<img src="./images/you_know_what_this_is_like.png" width="600" />

According to the Silicon Valley-based career guidance service Pathrise, [job seekers who sent 20+ job applications every week got more interviews and landed a job sooner](https://www.pathrise.com/guides/how-long-does-it-take-to-find-a-job/).

That’s a lot of work, especially since the general advice is to [customize your résumé for every job application](https://resume.io/blog/customize-resume-for-each-application) and to [write a custom cover letter for each job application](https://novoresume.com/career-blog/do-i-need-a-cover-letter) as well.

Now imagine going through that process at least 20 times a week.

This application sets out to solve that problem. It uses an AI — more specifically, a large language model (LLM) — to perform the repetitive task of creating résumés and cover letters that are customized for each job application you fill out.

## Setup

### Set up an OpenAI secret key
Because getting everyone in this meetup to set up and OpenAI API account and an API key would probably take all the allotted time, we’re going to keep things simple. **In this meetup, you’ll use Joey’s OpenAI account and an API key that will be active only during the meetup.** In order to run this application after the meetup, you’ll need the following:

- **An OpenAI account.** You can sign up for one at the [OpenAI Platform page](https://platform.openai.com/).
- **A secret key.** Log into OpenAI and visit the [API keys page](https://platform.openai.com/api-keys), and create a new secret key by clicking the **Create New Secret Key** button.
- **Some money in your OpenAI credit balance.** As it says on the [Billing page](https://platform.openai.com/settings/organization/billing/overview), “When your credit balance reaches $0, your API requests will stop working.” Fortunately, one or two US dollars should be enough to cover running this application many, many times.

Once you have created a secret key, copy that key. Create a file named `.env` in the same directory as this notebook and enter the following into that file:

```
OPENAI_API_KEY = replace_this_with_your_secret_key
```

> Filenames that begin with `.` (period) characters are system files, and for your safety, macOS hides them. You can make them visible by using the ⌘-shift-. keyboard shortcut (that is, hold down command, shift, and the `.` keys at the same time) when looking at a Finder window or when opening files.

Replace `replace_this_with_your_secret_key` with your secret key and save the file. By storing the API key in a `.env` file (and making sure that the .env is _not_ added to source control), you avoid “hard-coding” the API key into your application, which makes it more likely that someone unauthorized will discover it.

### Install the packages used by this application
There’s no need to reinvent the wheel when we Python has a large collection of packages to take care of things so we don’t have to code them ourselves. Let’s install the following packages:

- `dotenv`: Provides functions for reading `.env` files to create environment variables. We’ll use this to get the API keys so that the application can talk to LLMs.
- `markdown-pdf`: Converts Markdown documents into PDF documents. We’ll use this to generate the final copy of the customized résumé.
- `openai`: Provides functionality for communicating with OpenAI’s AI models.
- `pyperclip`: Gives our application access to the clipboard.

The command below installs these packages, which are listed in the file `requirements.txt`. If it doesn’t work, try replacing `pip` with `pip3`:

In [ ]:
! pip install -r requirements.txt

### Import the necessary packages
With all the packages installed, we can now import them, along with some standard Python packages, namely `os` and `sys`.

In [1]:
import os
import sys

import pyperclip
from dotenv import load_dotenv
from markdown_pdf import MarkdownPdf, Section
from openai import OpenAI

## Create functions for working with OpenAI

### Create an OpenAI client object
The first step in building an OpenAI-powered application is to create an instance of the OpenAI client class. We’ll call it `client`, and it will enable access to OpenAI’s various APIs, including the one we’ll use: GPT.

In [2]:
# When called without arguments, OpenAI will try to get
# the API key from the OPENAI_API_KEY environment variable.
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client = OpenAI()

### Create a completion function
Once we’ve created an OpenAI client object, let’s use it to create a _completion_. In the context of an LLM, a completion is the text that the model generates in response to a given prompt. It's essentially the output that the AI produces to “complete” the input it has been given.

In order to create a completion, we’ll use the `OpenAI` class’ `chat.completions.create()` method and provide it with the following information:

- **Model:** For the purposes of the meetup, we’ll use the **GPT-4o mini** model, a smaller, faster model that costs less to run. After the meetup, and once you’re using your own OpenAI account, you can change this out for a more capable (but more expensive) model.
- **Temperature:** A parameter that controls the randomness or predictability of the model's outputs. It's named after the temperature parameter in statistical physics, where higher temperatures lead to more disorder. It’s essentially a "creativity dial" that lets developers or users control how much the LLM sticks to the most likely responses versus exploring more varied possibilities. In most LLMs, this is a value between 0 and 2, and the default value is 1.
- **Messages:** A list of messages that make up the exchange or “conversation” with the LLM. Each message is a dictionary with two keys:
  - `role`: Defines the “speaker” of the message. It's essentially a label that identifies the source and purpose of each message in the exchange. It can be one of these three values:
    - `"system"`: The initial instructions or context given to the model. Sets the overall behavior, personality, and constraints for the LLM. Establishes rules and guidelines for how the model should respond. Often includes information about the model's capabilities and limitations. Not directly visible to users in most interfaces.
    - `"user"`: Represents the person interacting with the LLM. Provides queries, instructions, or content for the model to respond to. Drives the direction of the conversation.
    - `"assistant"`: Represents the LLM's responses. Contains the generated completions from the model. Follows the guidelines established in the system role, and responds directly to user inputs.

In [4]:
def create_completion(
    system_prompt,
    user_prompt,
    model="gpt-4o-mini",
    temperature=1.0
):
    completion = client.chat.completions.create(
        model=model,
        temperature=temperature,
        messages=[
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": user_prompt,
            },
        ],
    )
    return completion

In our test completion, let’s ask what the fastest bird in the world is.

In [5]:
test_completion = create_completion(
    "You are a helpful assistant",
    "What’s the fastest bird in the world?",
)

Now that we have the completion, let’s see what it contains.

In [6]:
print(test_completion)

ChatCompletion(id='chatcmpl-Bgl43O3DdIFcQ4ioHQZLpUs1F22BJ', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The fastest bird in the world is the peregrine falcon. During its hunting stoop (high-speed dive), it can reach speeds of over 240 miles per hour (386 kilometers per hour). This makes it not only the fastest bird but also the fastest animal on the planet. When flying level, the fastest bird is the common swift, which can reach speeds of around 69 miles per hour (111 kilometers per hour).', role='assistant', function_call=None, tool_calls=None, refusal=None, annotations=[]))], created=1749530391, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_34a54ae93c', usage=CompletionUsage(completion_tokens=86, prompt_tokens=25, total_tokens=111, prompt_tokens_details={'cached_tokens': 0, 'audio_tokens': 0}, completion_tokens_details={'reasoning_tokens': 0, 'audio_tokens': 0, 'accepte

The `create_completion()` function returns _a lot_ of information. Let’s update it so that instead of returning a `ChatCompletion` object, it returns just the info we _really_ care about: the answer.

`ChatCompletion` objects contain a list called `choices`, which contain the answers generated by the LLM. Most of the time, there’s just one item in `choices`, so we’ll extract that item, `choice[0]`. `choice[0]` has a `message` property containing the answer, and the text of that answer is in the `message` property’s `content` property.

Here’s an updated version of `create_completion()` that returns just the answer, and nothing else:

In [7]:
def create_completion(
    system_prompt,
    user_prompt,
    model="gpt-4o-mini",
    temperature=1.0
):
    completion = client.chat.completions.create(
        model=model,
        temperature=temperature,
        messages=[
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": user_prompt,
            },
        ],
    )
    return completion.choices[0].message.content

Let’s test this new version of `create_completion()`. It should now return the LLM’s answer, and nothing else.

In [8]:
test_completion = create_completion(
    "You are a helpful assistant",
    "What’s the fastest bird in the world?",
)
print(test_completion)

The fastest bird in the world is the peregrine falcon. When in a dive, known as a stoop, it can reach speeds of over 240 miles per hour (386 kilometers per hour). This incredible speed makes the peregrine falcon not only the fastest bird but also the fastest animal on the planet.


Now that we have a function to communicate with the LLM, let’s build the rest of the program.

## Define the inputs

We need to create three files that our app will use to generate resumes and cover letters.

### Reference résumé
In order to create résumés customized to fit job descriptions, the app needs a reference résumé. This reference résumé is simply your résumé in Markdown format, saved as a file named `reference_resume.md` in the `input` directory. For the meetup, we used my résumé for this file, which is included in this project’s repo.

By providing your résumé in Markdown format, you can specify formatting such as headings, bold and italics, bullet points, and even hyperlinks. This will be useful later, when we convert the resulting customized résumé into a PDF or Microsoft Word file.

### Job description
Copy the job description for the job you want to generate a customized résumé for and save it into a file named `job_description.md`. Convert any formatting in the job description into Markdown — the formatting can provide the LLM with additional context that it might use when generating a customized résumé. I provided a sample version of `job_description.md` in this project’s repo, in the `input` directory.

### System prompt
To provide better résumés and cover letters, this app will use a system prompt that defines the general way in which the LLM should behave when generting them. It’s in the `system_prompt.md` file in the `input` directory.

Think of a system prompt like this:

1. **It's like a job description and behavioral guideline:** The system prompt tells the AI who it is, how it should behave, what tone to use, what it can and cannot discuss, and how to handle different types of questions.
2. **It provides background knowledge:** The system prompt may include specific facts that the AI should know about itself or various topics.
3. **It establishes boundaries:** The prompt defines what kinds of requests the AI should decline and how to politely redirect conversations when needed.
4. **It shapes the AI's "personality":** Whether the AI comes across as formal, casual, detailed, concise, humorous, or serious is largely influenced by the system prompt.

When you interact with an AI assistant, you're seeing the result of these behind-the-scenes instructions working together with the AI's training. The system prompt is invisible to users but shapes every response you receive.

I’ve provided this prompt in a file named `system_prompt.md`. Take a look at it, and feel free to change it as you see fit.

### Reading the files
Let’s define `read_file()`, a function that returns the contents of a given text file...

In [9]:
def read_file(filename):
    try:
        with open(filename) as f:
            print(f"Reading {filename}...")
            return f.read()
    except FileNotFoundError:
        print(f"{filename} not found.")
        sys.exit(1)
    else:
        print(f"Successfully read {filename}.")

...and then use it to define the inputs for the app:

In [10]:
REFERENCE_RESUME = read_file("./input/reference_resume.md")
SYSTEM_PROMPT = read_file("./input/system_prompt.md")
JOB_DESCRIPTION = read_file("./input/job_description.md")

Reading ./input/reference_resume.md...
Reading ./input/system_prompt.md...
Reading ./input/job_description.md...


## Create functions to build the résumé

### Create a function to define a user prompt for optimizing the user’s résumé
The _user prompt_ is the input or question that the user sends to the LLM. Let’s define a function that takes the user’s résumé and a job description and returns a user prompt asking the LLM to create a version of the résumé optimized for the given job description.

In [11]:
def create_resume_prompt(reference_resume, job_description):
    return f"""
        I need you to optimize a résumé to better match a job description. The goal is to present the candidate's experience 
        in a way that shows they're an excellent fit for the position, while remaining truthful and professional.

        Here's the original résumé in Markdown format...
        --- BEGIN RESUME ---
        {reference_resume}
        --- END RESUME ---
        
        ...and here's the job description that the résumé should be tailored to:
        --- BEGIN JOB DESCRIPTION ---
        {job_description}
        --- END JOB DESCRIPTION ---

        Please analyze the job description, identify key requirements and skills, and modify the résumé to highlight 
        relevant experience and accomplishments that match these requirements.

        In your response, please ensure the following:
        1. Maintain the original formatting and structure of the résumé.
        2. Do not invent any qualifications or experiences that are not present in the original résumé.
        3. Focus on emphasizing relevant skills and experiences.
        4. Use strong action verbs and quantify achievements where possible.
        5. Ensure the résumé is clear, concise, and professional.
        6. Maintain the Markdown formatting in your response.
        7. Return ONLY the optimized résumé, without any explanations.

        Return the optimized resume in Markdown format.
    """

### Create a function to generate a customized resume

This function will take the following inputs:

1. Base résumé
2. Job description
3. System prompt
4. LLM model (defaults to `"gpt-4o-mini"`)
5. Temperature (defaults to `1.0`)

It outputs a new résumé that is based on the base résumé, but fine-tuned to match the job description, in Markdown format. 

In [12]:
def generate_customized_resume(
    reference_resume,
    job_description,
    system_prompt,
    model="gpt-4o-mini",
    temperature=1.0
):
    return create_completion(
        system_prompt,
        create_resume_prompt(reference_resume, job_description),
        model,
        temperature
    )

Now that we have the `generate_customized_resume()` function, let’s run it and store the result in a variable named `OPTIMIZED_RESUME`. Be patient when you run it; the process will typically take anywhere from 10 to 20 seconds, but can sometimes take longer.

Once we have the generated résumé, we’ll write it to a file named `optimized_resume.md` in the `output` directory.

In [13]:
OPTIMIZED_RESUME = generate_customized_resume(
    REFERENCE_RESUME, JOB_DESCRIPTION, SYSTEM_PROMPT
)
with open(f"./output/optimized_resume.md", "w") as f:
    f.write(OPTIMIZED_RESUME)

## Cover letter

### Create a function to define a user prompt for creating a cover letter
Let’s define a function that returns a user prompt asking the LLM to create a cover letter, given the following information:

- `resume`: The résumé on which the cover letter will be based. You could use the reference résumé, but it would be far better if you used the optimized one, since it’s what you’re going to provide with the cover letter.
- `job_description`: The same job description you used for the résumé.
- `company_name`: The name of the company to whom you’re sending the cover letter. The LLM will incorporate the company name into the cover letter.
- `recipient_name`: Optional — if you know the recipient’s name, you can provide it, and the LLM will incorporate it into the cover letter.
- `additional_context`: If there’s anything that the LLM should know about you, your experience, or your skills that you think should be mentioned in the cover letter, you can provide this information.

In [14]:
def create_cover_letter_prompt(
    resume, 
    job_description, 
    company_name, 
    recipient_name="Unknown", 
    additional_context=""
):
    return f"""
        Context
        -------
        You are tasked with generating a highly personalized cover letter that connects a candidate's qualifications
        to a specific job opportunity. Use the provided resume and job description to create a persuasive, professional
        letter that positions the candidate as an ideal match.

        Input Information
        -----------------
        Job Description:
        {job_description}
        
        Resume:
        {resume}
        
        - Company Name: {company_name}
        - Recipient Name: {recipient_name}
        - Additional Context: 
          {additional_context}
        
        Instructions
        ------------
        Create a tailored cover letter that:

        - Opens with a compelling introduction that mentions the specific position and briefly explains the candidate's interest
        - Identifies 2-3 key job requirements and directly connects them to specific experiences or skills from the resume
        - Demonstrates understanding of the company's mission, values, or recent achievements (if provided)
        - Explains why the candidate would be an excellent cultural fit based on their background and the company's environment
        - Closes with enthusiasm for the opportunity and a clear call to action

        Requirements
        ------------
        - Maintain a professional yet conversational tone appropriate for the industry and position level
        - Be concise (350-450 words) and focused only on the most relevant qualifications
        - Use concrete examples and metrics from the resume when applicable
        - Avoid generic template language and empty flattery
        - Structure with proper business letter formatting including date, addresses, greeting, and signature
        - Match the candidate's experience level in language and tone (entry-level, mid-career, or executive)
        - Emphasize transferable skills when direct experience is limited
        - Include only truthful information present in the resume

        Output Format
        -------------
        Return a properly formatted business letter in Markdown that includes:

        - Current date
        - Recipient's information (if provided)
        - Professional greeting
        - 3-4 well-structured paragraphs
        - Professional closing
        - Candidate's name as signature

        The letter should read as if written by the candidate themselves, reflecting their genuine interest and qualifications.
    """

### Create a function to generate a cover letter

Now that we have a cover letter prompt function, let’s create a function to actually generate the cover letter. Just as `generate_customized_resume()` is a wrapper around the `create_completion()` function to generate a résumé, this function, `generate_cover_letter()`, is the cover letter version.

It takes the following inputs:

1. Résumé (preferably the fine-tuned one)
2. Job description
3. Company name
4. Recipient name (defaults to `"Unknown"`)
5. Additional context (defaults to `""`)
6. System prompt (defaults to `SYSTEM_PROMPT`)
7. LLM model (defaults to `"gpt-4o-mini"`)
8. Temperature (defaults to `1.0`)

It outputs a cover letter.

In [15]:
def generate_cover_letter(
    resume,
    job_description,
    company_name,
    recipient_name="Unknown", 
    additional_context="",
    system_prompt=SYSTEM_PROMPT,
    model="gpt-4o-mini",
    temperature=1.0
):
    return create_completion(
        system_prompt,
        create_cover_letter_prompt(resume, job_description, company_name, recipient_name, additional_context),
        model,
        temperature
    )

Let’s run the `generate_cover_letter()` function and store its result in a variable named `COVER_LETTER`. Just like with the optimized résumé, it will take about 10 - 20 seconds to generate the cover letter.

Once the cover letter has been generated, we’ll write it to a file named `cover_letter.md` in the `output` directory.

In [16]:
COVER_LETTER = generate_cover_letter(
    OPTIMIZED_RESUME, 
    JOB_DESCRIPTION, 
    "Datadog", 
    None,
    "I have 10+ years experience in Python programming.")
with open(f"./output/cover_letter.md", "w") as f:
    f.write(COVER_LETTER)

## Copy the documents to the clipboard

If you’ve run every cell in this notebook up to this point, the application has the following text data in Markdown format:

- The optimized résumé, which is stored in memory in the “constant” `OPTIMIZED_RESUME`
- The cover letter, which lives in the “constant” `COVER_LETTER`

You could immediately turn both of these into PDF files and submit them, but I don’t recommend that. You’ll want to review them first, and perhaps make some edits and fix some quirks that give away that your résumé and cover letter were generated.

### Copy the optimized résumé to the clipboard
The code cell below copies the optimized résumé to the clipboard. Run it...

In [17]:
pyperclip.copy(OPTIMIZED_RESUME)

...then paste the clipboard’s contents into an editor that will properly format Markdown text that you paste into it, such as Google Docs.

### Copy the cover letter to the clipboard
The code cell below copies the cover letter to the clipboard. As with the optimized résumé, run the cell below...

In [ ]:
pyperclip.copy(COVER_LETTER)

...then paste the clipboard’s contents into Google Docs or any other editor that will properly format Markdown text that you paste into it.